In [ ]:
'''
[EDA] -> 병건님
Label 별로 plot을 그려보고
fft 함수를 사용해서 그려보고 -> 이 둘을 비교 
kurtosis, rms (scipy 라이브러리와 함수를 사용해서 이또한 그려보기) 적용하여 라벨별로 hist 그려보기

1. 정윤님께서, 주신 데이터 주파수나 몇초동안 측정 된건지 파악하기
-> 1초동안 측정 된 것이라고함

2. 데이터의 col 수와 hz 값? 을 나누었을 때 나오는 값이 시간임
    ex) col : 8000 , hz = 8000 -> 1 sec 동안 측정 된 데이터
    
3. 정윤님 요청
 정상 모델이 비정상을 잘 분류 하는지? -> 정확도
 비정상 label 군집이 잘 잡히는지?
'''

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from process_signal import positive_fft, rms
from scipy.stats import skew, kurtosis

In [ ]:
import pandas as pd
df = pd.read_csv('sample.csv')
df

In [ ]:
label_lst = df.label.unique().tolist()
label_lst

In [ ]:
zero = df[df.label == 0]
one = df[df.label == 1]
two = df[df.label == 2]
three = df[df.label == 3]
four = df[df.label == 4]

label_df = [zero, one, two, three, four]

for data in enumerate(label_df):
    print(f"label : {data[0]}, size : {len(data[1])}")
    

In [ ]:
'''
practice
'''
df.loc[df.label == 0].iloc[:,2:-1]
label_0_ary = df.loc[df.label == 0].iloc[:,2:-1].values
label_0_ary

In [ ]:
'''
label별로 구분 , 0~8000 Hz까지 신호 
'''
label_0_samples = df.loc[df.label == 0].iloc[:,2:-1].values
label_1_samples = df.loc[df.label == 1].iloc[:,2:-1].values
label_2_samples = df.loc[df.label == 2].iloc[:,2:-1].values
label_3_samples = df.loc[df.label == 3].iloc[:,2:-1].values
label_4_samples = df.loc[df.label == 4].iloc[:,2:-1].values

print(len(label_4_samples))

In [ ]:
'''
raw signal visualization
'''
import matplotlib.pyplot as plt

num_subplots = int(len(label_0_samples) / 2 - 1)
fig, axes = plt.subplots(num_subplots, 1, figsize=(10, 2*num_subplots))

for n in range(num_subplots):
    axes[n].plot(label_0_samples[n])
plt.subplots_adjust(hspace=0.5)
plt.show()

In [ ]:
'''
fft랑 비교 (label 0 ~ 4 까지 한큐에)
'''
import numpy as np
from process_signal import positive_fft
import matplotlib.pyplot as plt

sample_lst = [label_0_samples,label_1_samples, label_2_samples, label_3_samples, label_4_samples]
label_num = 0

for sample in sample_lst:
    num_samples = len(sample)
    fig, axes = plt.subplots(num_samples, 2, figsize=(15, 5*120))

    for n in range(num_samples):
        axes[n, 0].plot(sample[n])
        axes[n, 0].set_title(f'Label {label_num} Sample {n} RMS {round((np.mean(sample[n]**2))**0.5,4)}')
        
        freq, mag = positive_fft(sample[n], 8000)  #positive_fft 함수 두번째 인자 1맞는가?
        axes[n, 1].plot(freq, mag)
        axes[n, 1].set_title(f'FFT of Sample {n}')

    label_num += 1
     
    plt.subplots_adjust(wspace=0.3, hspace=0.5)
    plt.show()

In [ ]:
'''
위에 시각화로 알게된 점
Label = 0 (이상) 의 경우 raw signal의 파형이 예뻐서 FFT를 수행해도 결과가 이쁘게 나온다.

하지만, 비정상 SIGNAL의 경우 파형이 일부 이쁘지 않은 것을 확인할 수 있었다.
    여기서, Label2와 Label4가 Label3에 비해서도 raw signal 파형이 이쁘지 않다. (특히 2가 심하다.)
        label2, 4와 3을 비교해보면 어떨까
'''

In [ ]:
'''
Label 0 일때, fft 랑 비교
'''
num_samples = len(label_0_samples)
fig, axes = plt.subplots(num_samples, 2, figsize=(15, 5*120))

for n in range(num_samples):
    axes[n, 0].plot(label_0_samples[n])
    axes[n, 0].set_title(f'Label 0 Sample {n}')
    
    freq, mag = positive_fft(label_0_samples[n], 1)
    axes[n, 1].plot(freq, mag)
    axes[n, 1].set_title(f'FFT of Sample {n}')

plt.subplots_adjust(wspace=0.3, hspace=0.5)
plt.show()

In [ ]:
'''
각 label 당 한 sample (한 row) 을 kurtosis, rms에  넣었을 때
나오는 return 값들을 받아서, 각 라벨 별로 히스토그램을 그려서 분포를 한번 비교해보기
'''
sample_lst = [label_0_samples,label_1_samples, label_2_samples, label_3_samples, label_4_samples]
label_cnt = 0

fig2, axes2 = plt.subplots(len(sample_lst), 2, figsize = (15, 30))

for sample in sample_lst:
    rms_values_lst = []
    kurtosis_values_lst = []
    
    for n in range(len(sample)):
        rms_values_lst.append(rms(sample[n]))
        kurtosis_values_lst.append(kurtosis(sample[n]))

    axes2[label_cnt,0].hist(rms_values_lst)
    axes2[label_cnt,0].set_title(f'Label {label_cnt} rms Histogram')
    
    axes2[label_cnt,1].hist(kurtosis_values_lst)
    axes2[label_cnt,1].set_title(f'Label {label_cnt} kurtosis Histogram')
    
    label_cnt += 1

In [ ]:
df

In [ ]:
'''
RMS, Kurtosis 적용 후, Label 별로 비교하지 않고 한꺼번에 비교하기 위하여 다르게 시각화를 해보았다.
'''

import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import kurtosis

# Assuming rms is a function that calculates the root mean square of a sample
def rms(sample):
    return np.sqrt(np.mean(np.square(sample)))

# Sample data
sample_lst = [label_0_samples, label_1_samples, label_2_samples, label_3_samples, label_4_samples]

# Define colors for each label
colors = ['blue', 'orange', 'green', 'red', 'purple']
labels = [f'Label {i}' for i in range(len(sample_lst))]

fig, axes = plt.subplots(1, 2, figsize=(15, 7))

# Plot RMS histograms
for idx, (sample, color, label) in enumerate(zip(sample_lst, colors, labels)):
    rms_values = [rms(s) for s in sample]
    axes[0].hist(rms_values, bins=20, color=color, alpha=0.5, label=label)

# Set titles and legend for RMS histogram
axes[0].set_title('RMS Histogram')
axes[0].set_xlabel('RMS Value')
axes[0].set_ylabel('Frequency')
axes[0].legend()

# Plot Kurtosis histograms
for idx, (sample, color, label) in enumerate(zip(sample_lst, colors, labels)):
    kurtosis_values = [kurtosis(s) for s in sample]
    axes[1].hist(kurtosis_values, bins=20, color=color, alpha=0.5, label=label)

# Set titles and legend for Kurtosis histogram
axes[1].set_title('Kurtosis Histogram')
axes[1].set_xlabel('Kurtosis Value')
axes[1].set_ylabel('Frequency')
axes[1].legend()

plt.tight_layout()
plt.show()


In [ ]:
'''
알 수 있는점?
RMS 값에서 가장 작은 값들의 분포는 역시 Label 2 가 제일 많았고,
Kurtosis 값 또한, Label2 에서 이상치들이 많이 존재하는 것처럼 보인다.

Label2 / 3 / 4 중에 Label 2 에 부여된 값들을 분석을 해보면 좋을것 같다.
'''

In [ ]:
'''
3. 정윤님 요청
 정상 모델이 비정상을 잘 분류 하는지?
 비정상 label 군집이 잘 잡히는지?
'''

In [ ]:
'''
MAX <2 인 값은 안봐도 되신다고 하셨다
'''
dff = df[df.MAX>2]
dff.drop_duplicates(subset=None, keep='first', inplace=True, ignore_index=False)
dff.reset_index(drop = True, inplace = True)
dff

In [ ]:
X = dff.drop(columns = ['id', 'label', 'MAX'])
y = dff['label']

X

In [ ]:
# 0 말고 다른 label 은 모두 1처리
y_binary = y.apply(lambda x: 0 if x == 0 else 1)
y_binary

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score

# train / test
X_train, X_test, y_train, y_test = train_test_split(X, y_binary, test_size=0.2, random_state=42)

# 학습
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train, y_train)

# 예측
y_pred = clf.predict(X_test)

# 성능 평가
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print(f'F1 Score: {f1}')


In [ ]:
X_anomalous = X[y_binary == 1]
print(f'비정상 레이블 수 : {len(dff) - len(dff[dff.label==0])}')
X_anomalous

In [ ]:
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

# 비정상 데이터만 추출합니다
X_anomalous = X[y_binary == 1]

# PCA
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_anomalous)

# K-means 
kmeans = KMeans(n_clusters=4, random_state=42)
clusters = kmeans.fit_predict(X_pca)

# Visualization
plt.scatter(X_pca[:, 0], X_pca[:, 1], c=clusters, cmap='viridis')
plt.xlabel('PCA Component 1')
plt.ylabel('PCA Component 2')
plt.title('Cluster Visualization of Anomalous Data')
plt.colorbar()
plt.show()


In [ ]:
'''
3차원
'''
from mpl_toolkits.mplot3d import Axes3D

X_anomalous = X[y_binary == 1]

pca = PCA(n_components=3)
X_pca = pca.fit_transform(X_anomalous)


kmeans = KMeans(n_clusters=4, random_state=42)
clusters = kmeans.fit_predict(X_pca)

# 3D 시각화
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')
# 군집별 색 다르게 시각화
scatter = ax.scatter(X_pca[:, 0], X_pca[:, 1], X_pca[:, 2], c=clusters, cmap='viridis')

ax.set_xlabel('PCA Component 1')
ax.set_ylabel('PCA Component 2')
ax.set_zlabel('PCA Component 3')
ax.set_title('3D Cluster Visualization of Anomalous Data')

cbar = plt.colorbar(scatter, ax=ax)
cbar.set_label('Cluster')

plt.show()


In [ ]:
'''
비정상 label 값 처리를 1로하지 X
'''
import pandas as pd

# 비정상 데이터만 추출 (label이 1~4인 데이터)
X_anomalous = X[y > 0]
y_anomalous = y[y > 0]


In [ ]:
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

# PCA
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_anomalous)

# K-means 군집화
kmeans = KMeans(n_clusters=4, random_state=42)
clusters = kmeans.fit_predict(X_pca)

# 시각화
plt.figure(figsize=(12, 8))
scatter = plt.scatter(X_pca[:, 0], X_pca[:, 1], c=clusters, cmap='viridis', label=clusters)
plt.xlabel('PCA Component 1')
plt.ylabel('PCA Component 2')
plt.title('Cluster Visualization of Anomalous Data')

# 레이블에 따라 다른 색으로 시각화
for i in range(1, 5):
    mask = y_anomalous == i
    plt.scatter(X_pca[mask, 0], X_pca[mask, 1], label=f'Label {i}')


plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.colorbar(scatter, label='Cluster')
plt.tight_layout(rect=[0, 0, 0.85, 1])
plt.show()
